In [1]:
import matplotlib.pyplot as plt

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

In [20]:
from vertex import BiomeCollection
from nosie_functions import NOISE_FUNCTIONS, range_field
import numpy as np

collection = BiomeCollection.load()
base_field = range_field(collection.world_params)



In [17]:
from tqdm import tqdm

vector_fields = {}

for field in tqdm(collection.vector_fields):
    func = NOISE_FUNCTIONS[field.noise_type]
    vector_eval = np.vectorize(func(**field.noise_params))
    noise_map = vector_eval(base_field.copy())

    # normalize
    nm_min = noise_map.min()
    nm_range = noise_map.max() - nm_min
    noise_map -= noise_map.min()
    noise_map /= nm_range

    # map
    noise_map *= (field.norm_max - field.norm_min)
    noise_map += field.norm_min

    vector_fields[field.name] = noise_map

100%|██████████| 1/1 [00:10<00:00, 10.52s/it]


In [27]:
vector_fields[field.name].max()

1000.0

In [41]:
import numpy as np
from nosie_functions import NOISE_FUNCTIONS, range_field
from vertex import BiomeCollection
from collections import defaultdict

resultant_fields = defaultdict(list)

collection = BiomeCollection.load()

for rule in collection.rules:
    eval_field = vector_fields[rule.vector_field]
    print(eval_field.min(), eval_field.max())
    resultant_fields[rule.property_name].append({
        "field": rule.evaluate(eval_field),
        "weight": rule.weight,
    })
    print(resultant_fields[rule.property_name][-1]['field'].min())
    print(resultant_fields[rule.property_name][-1]['field'].max())
    print(resultant_fields[rule.property_name][-1]['field'].mean())


0.0 1000.0
33.0
33.0
33.0


In [30]:
resultant_fields['temperature_f'][0]['field'].min()

99.0

In [10]:
# fig = plt.figure(figsize=(40, 40), dpi=20, facecolor='w', edgecolor='k')
# plt.imshow(pic, cmap='gray')
# plt.show()